# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [31]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format,dayofweek
from pyspark.sql.functions import expr
from pyspark.sql.functions import unix_timestamp,from_unixtime
from pyspark.sql.types import IntegerType,TimestampType


In [5]:
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [5]:
# Read in the data here


In [6]:
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [53]:
df_airport=spark.read.csv("airport-codes_csv.csv",header=True)

In [54]:
df_airport.show(10)
df_airport.count()

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|     US-AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|
| 00AL|small_airport|        Epps Airpark|         820|       NA|         US|     

55075

In [60]:
df_city=spark.read.option("header", True).option("delimiter", ";").csv("us-cities-demographics.csv")

In [61]:
df_city.show(5)
df_city.count()

+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|            City|        State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race|Count|
+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|   Silver Spring|     Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|  Hispanic or Latino|25924|
|          Quincy|Massachusetts|      41.0|          44129|            49500|           93629|              4147|       32935|                  2.39|        MA|               White|58723|
|          Hoover|      Alabama|      38.5|          38040| 

2891

In [6]:
#df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')
#df_spark.write.parquet("sas_data")

In [7]:
#write to parquet
df_spark=spark.read.parquet("sas_data")

In [40]:
df_spark.show(1)
df_spark.take(1)
df_spark.head()
df_spark.count()

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|5748517.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     CA|20582.0|  40.0|    1.0|  1.0|20160430|     SYD| null|      G|      O|   null|      M| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+--

3096313

In [41]:

df_spark.head()

Row(cicid=5748517.0, i94yr=2016.0, i94mon=4.0, i94cit=245.0, i94res=438.0, i94port='LOS', arrdate=20574.0, i94mode=1.0, i94addr='CA', depdate=20582.0, i94bir=40.0, i94visa=1.0, count=1.0, dtadfile='20160430', visapost='SYD', occup=None, entdepa='G', entdepd='O', entdepu=None, matflag='M', biryear=1976.0, dtaddto='10292016', gender='F', insnum=None, airline='QF', admnum=94953870030.0, fltno='00011', visatype='B1')

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [16]:
df_spark.select("cicid").distinct().count()

3096313

In [33]:
df_spark.groupby("i94port").count().orderBy(col("count"),ascending=False).show()

+-------+------+
|i94port| count|
+-------+------+
|    NYC|485916|
|    MIA|343941|
|    LOS|310163|
|    SFR|152586|
|    ORL|149195|
|    HHW|142720|
|    NEW|136122|
|    CHI|130564|
|    HOU|101481|
|    FTL| 95977|
|    ATL| 92579|
|    LVG| 89280|
|    AGA| 80919|
|    WAS| 74835|
|    DAL| 71809|
|    BOS| 57354|
|    SEA| 47719|
|    PHO| 38890|
|    DET| 37832|
|    TAM| 25632|
+-------+------+
only showing top 20 rows



In [36]:
df_spark.groupby("visatype").count().orderBy(col("count"),ascending=False).show()

+--------+-------+
|visatype|  count|
+--------+-------+
|      WT|1309059|
|      B2|1117897|
|      WB| 282983|
|      B1| 212410|
|     GMT|  89133|
|      F1|  39016|
|      E2|  19383|
|      CP|  14758|
|      E1|   3743|
|       I|   3176|
|      F2|   2984|
|      M1|   1317|
|      I1|    234|
|     GMB|    150|
|      M2|     49|
|     SBP|     11|
|     CPL|     10|
+--------+-------+



In [39]:
df_spark.groupby("i94mode").count().orderBy(col("count"),ascending=False).show()

+-------+-------+
|i94mode|  count|
+-------+-------+
|    1.0|2994505|
|    3.0|  66660|
|    2.0|  26349|
|    9.0|   8560|
|   null|    239|
+-------+-------+



In [38]:
df_spark.groupby("visapost").count().orderBy(col("count"),ascending=False).show()

+--------+-------+
|visapost|  count|
+--------+-------+
|    null|1881250|
|     MEX|  84720|
|     SPL|  65678|
|     BNS|  62032|
|     GUZ|  48298|
|     BGT|  46074|
|     CRS|  37137|
|     BEJ|  36703|
|     SHG|  35507|
|     GDL|  30970|
|     RDJ|  29943|
|     TLV|  28903|
|     BMB|  28108|
|     MDR|  26497|
|     GYQ|  26231|
|     SDO|  20924|
|     MNL|  19513|
|     MTR|  18105|
|     LMA|  17479|
|     SNJ|  16717|
+--------+-------+
only showing top 20 rows



In [42]:
df_spark.groupby("gender").count().orderBy(col("count"),ascending=False).show()

+------+-------+
|gender|  count|
+------+-------+
|     M|1377224|
|     F|1302743|
|  null| 414269|
|     X|   1610|
|     U|    467|
+------+-------+



In [44]:
df_spark.groupby("biryear").count().orderBy(col("biryear")).show(100)

+-------+-----+
|biryear|count|
+-------+-----+
|   null|  802|
| 1902.0|    1|
| 1905.0|    1|
| 1906.0|    1|
| 1907.0|    2|
| 1908.0|    2|
| 1909.0|    1|
| 1911.0|    2|
| 1913.0|    1|
| 1914.0|    4|
| 1915.0|    2|
| 1916.0|   24|
| 1917.0|   19|
| 1918.0|   26|
| 1919.0|   52|
| 1920.0|   46|
| 1921.0|   88|
| 1922.0|  104|
| 1923.0|  185|
| 1924.0|  241|
| 1925.0|  319|
| 1926.0|  463|
| 1927.0|  638|
| 1928.0|  884|
| 1929.0| 1204|
| 1930.0| 1594|
| 1931.0| 1999|
| 1932.0| 2500|
| 1933.0| 2965|
| 1934.0| 3784|
| 1935.0| 4629|
| 1936.0| 5635|
| 1937.0| 6813|
| 1938.0| 8019|
| 1939.0| 9340|
| 1940.0|10897|
| 1941.0|12305|
| 1942.0|14198|
| 1943.0|16238|
| 1944.0|18559|
| 1945.0|19988|
| 1946.0|24891|
| 1947.0|28451|
| 1948.0|29576|
| 1949.0|32063|
| 1950.0|33667|
| 1951.0|34141|
| 1952.0|37002|
| 1953.0|38333|
| 1954.0|41352|
| 1955.0|43914|
| 1956.0|45950|
| 1957.0|44921|
| 1958.0|45853|
| 1959.0|47674|
| 1960.0|49722|
| 1961.0|50288|
| 1962.0|53865|
| 1963.0|55673|
| 1964.0

In [48]:
df_airline=df_spark.groupby("airline").count().orderBy(col("count"),ascending=False)
df_airline.show(10)
df_airline.count()

+-------+------+
|airline| count|
+-------+------+
|     AA|310091|
|     UA|264271|
|     DL|252526|
|     BA|190997|
|     LH|120556|
|     VS|113384|
|   null| 83627|
|     AF| 81113|
|     KE| 71047|
|     JL| 69075|
+-------+------+
only showing top 10 rows



535

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.